# Import Packages

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, ArrayType
from pyspark.sql.functions import udf
from nltk.stem import SnowballStemmer
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from nltk.stem import WordNetLemmatizer
from pyspark.sql.functions import trim, lower, col
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from pyspark.sql.functions import udf
import pandas as pd


In [4]:
df = pd.read_csv('final_cleaned_df.csv',on_bad_lines='skip')

In [10]:
df.head(10).to_csv('snippet.csv')

Adding the final review csv file 

In [7]:
%cd Big_Data_IMDb/

# add the csv files here otherwise the code wont work, cannot push it to GitHub
# sadly, too large
!mkdir movie_reviews
%pwd

!mv /Users/bognarlili/Desktop/final_reviews_data.csv movie_reviews/

!ls movie_reviews/

/Users/bognarlili/Big_Data_IMDb/Big_Data_IMDb
mkdir: movie_reviews: File exists
mv: /Users/bognarlili/Desktop/final_reviews_data.csv: No such file or directory
final_reviews_data.csv


Loading CSV into Spark

In [2]:
spark = SparkSession.builder \
    .appName("IMDb Reviews Analysis") \
    .getOrCreate()

df = spark.read.csv("movie_reviews/final_reviews_data.csv", header=True, inferSchema=True)
df.show(5)  


+---------+------------------+--------+----+-----+------------------+--------------------+--------------------+------------+--------------------+
|   tconst|       movie_title|numVotes|year|label|tomatometer_status|         review_type|               genre|review_label|              review|
+---------+------------------+--------+----+-----+------------------+--------------------+--------------------+------------+--------------------+
|tt0009369|            mickey|  1119.0|1918|false|              NULL|                NULL|                NULL|        NULL|['Mack Sennett ha...|
|tt0010600|          the doll|  1898.0|1919| true|              NULL|                NULL|                NULL|        NULL|['"The Doll" is a...|
|tt0011439| the mark of zorro|  2439.0|1920| true|             Fresh|['Rotten', 'Fresh...|Action & Adventur...|       Fresh|['It was such a s...|
|tt0011607|the parson's widow|  1264.0|1920| true|              NULL|                NULL|                NULL|        NULL|

In [3]:
df.select('review').first()

Row(review='[\'Mack Sennett had a strong reputation for producing wild\', \'violent\', \'fast- paced slapstick that often got its laughs without even a superficial attempt to make sense. He got that reputation for the simple reason that it\'s true. However\', \'it\'s interesting to see how when Sennett knew he had on his hands a comedian whose laughs come from subtleties or reactions rather than fast antics\', \'he knows to slacken the pace. That was true with many of the brilliant Harry Langdon shorts he would produce later\', \'and it is true here in "Mickey" with Mabel Normand.Mabel is the star and it is she on which the movie turns. She steals every scene she appears in and has infinite screen magnetism\', \'with her attractive\', \'fascinating face\', \'constantly changing expression\', \'and childlike and uninhibited yet somehow ironic manner. The greatest moments of comedy come in little bits of performance\', \'as Mabel comes up with many ingenious ways to hide dust she has swe

# Cleaning

Remove entries with missing review data as these movies are not present in the IMDB database and therefore represent incomplete data points that would skew our analysis

In [4]:
# check the number of the rows
# remove entries with missing review data
# drop data before 1930
print(f"original nr of rows: {df.count()}")
df = df.dropna(subset=["review"])
df = df.filter(df["year"] >= 1930)
print(f"nr of rows after cleaning: {df.count()}")
df.show(5)


original nr of rows: 8061
nr of rows after cleaning: 7914
+---------+--------------------+--------+----+-----+------------------+-----------+-----+------------+--------------------+
|   tconst|         movie_title|numVotes|year|label|tomatometer_status|review_type|genre|review_label|              review|
+---------+--------------------+--------+----+-----+------------------+-----------+-----+------------+--------------------+
|tt0016029|  the little colonel|  1646.0|1935| true|              NULL|       NULL| NULL|        NULL|['THE LITTLE COLO...|
|tt0017961|           happiness|  1080.0|1935| true|              NULL|       NULL| NULL|        NULL|['I really love s...|
|tt0020298|         queen kelly|  3226.0|1932| true|              NULL|       NULL| NULL|        NULL|['I'd imagine tha...|
|tt0020768|           city girl|  3199.0|1930| true|              NULL|       NULL| NULL|        NULL|['Silent film may...|
|tt0021309|the story of the fox|     NaN|1937| true|              NULL|   

In [10]:
df.select('review').collect()[1][0]  # where row_number is the 0-indexed row you want (e.g., 2 for third row)

'[\'I really love silent cinema of all types\', \'and some of my very favorite films are silent (Voyage to the Moon\', \'Battleship Potemkin\', \'The Passion of Joan of Arc\', \'Our Hospitality\', \'Sherlock Jr.\', \'Safety Last\', \'City Lights\', \'Modern Times). Battleship Potemkin (and Sergei Eisenstein in general) got me interested in Soviet silent film\', \'so I was excited to find this particular title\', \'Happiness\', \'on my local video shelf (although that is not a link\', \'this is available for purchase\', \'if not at Amazon\', \'check other sites.  I picked it up and put it in my VCR. I had expected something heavy and powerful like Potemkin (and many other films that I had read about)\', \'but it turned out not to be exactly what I expected. Happiness is\', \'in fact\', \'a Russian silent comedy. No\', \'it is quite unlike the silent comedies of Keaton\', \'Lloyd\', \'and Chaplin. It shares a few characteristics (especially Chaplin\'s famed swift-kick-in-the-butt\', \'so

# Preparing for Sentiment Analysis

In [15]:
# Import necessary modules
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from pyspark.sql.types import ArrayType, StringType

class TextProcessor:
    """
    A class to handle text preprocessing steps including:
    - Tokenization (splitting text into words)
    - Stopword removal (removing common words like "the", "is", etc.)
    - Stemming (reducing words to their root form)
    - Lemmatization (reducing words to their base form)
    """

    def __init__(self):
        """
        Constructor that initializes the stemmer and lemmatizer objects.
        - SnowballStemmer: Used for stemming words.
        - WordNetLemmatizer: Used for lemmatizing words.
        """
        self.stemmer = SnowballStemmer(language="english")
        self.lemmatizer = WordNetLemmatizer()

    def tokenize(self, df, input_col, output_col):
        """
        Tokenizes the specified column into words using RegexTokenizer.

        """
        tokenizer = RegexTokenizer(
            inputCol=input_col, 
            outputCol=output_col, 
            pattern="\\W+",   # Split on non-word characters
            toLowercase=True  # Convert text to lowercase
        )
        return tokenizer.transform(df)
    
    def remove_stopwords(self, df, input_col, output_col):
        """
        Removes stopwords (e.g., "the", "and", "is") using StopWordsRemover.
            output_col (str): Name of the new column with stopwords removed.
        """
        remover = StopWordsRemover(
            inputCol=input_col,
            outputCol=output_col
        )
        return remover.transform(df)
    
    @staticmethod
    @udf(ArrayType(StringType()))
    def stemming(words):
        """
        Applies stemming using SnowballStemmer to reduce words to their root form.

        """
        if words is not None:
            stemmer = SnowballStemmer(language="english")
            return [stemmer.stem(word) for word in words]
        return None
    
    @staticmethod
    @udf(ArrayType(StringType()))
    def lemmatizing(words):
        """
        Applies lemmatization using WordNetLemmatizer to reduce words to their base form.

        """
        if words is not None:
            lemmatizer = WordNetLemmatizer()
            return [lemmatizer.lemmatize(word) for word in words]
        return None



# Processing Pipeline

In [17]:
#processor instance
processor = TextProcessor()

# Initialize SparkSession 
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IMDb Reviews Analysis") \
    .getOrCreate()

# Load CSV file into a PySpark DataFrame
df = spark.read.csv("movie_reviews/final_reviews_data.csv", header=True, inferSchema=True)

#cleaning pipeline in one call

def run_cleaning(df):
    """
    Executes all preprocessing steps:
    1. Tokenization
    2. Stopword removal
    3. Stemming
    4. Lemmatization
    
    Args:
        df (DataFrame): Input PySpark DataFrame.

    Returns:
        DataFrame: Processed DataFrame with cleaned text data.
    """
    processor = TextProcessor()
    
    # Execute steps in order
    df = processor.tokenize(df, input_col="review", output_col="review_words")
    df = processor.remove_stopwords(df, input_col="review_words", output_col="review_clean")
    df = df.withColumn("review_stemmed", processor.stemming(col("review_clean")))
    df = df.withColumn("review_lemmatized", processor.lemmatizing(col("review_stemmed")))

    return df

df_cleaned = run_cleaning(df)
df_cleaned.show(truncate=False)


Py4JJavaError: An error occurred while calling o268.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 1 times, most recent failure: Lost task 0.0 in stage 17.0 (TID 70) (10.30.1.148 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD.$anonfun$mapParti

In [16]:
file_path = "NLP_review_cleaning.ipynb"

# Open and close the file to save it
with open(file_path, "a") as f:
    pass

print(f"✅ {file_path} has been saved.")


✅ NLP_review_cleaning.ipynb has been saved.
